# Segmenting and Clustering Neighborhoods in Toronto 
### Peer-graded assignment in Applied Data Science Capstone by IBM/Coursera 

#### *Submitted by Jessiedee Mark B. Gingo*

## Table of contents
* [Installing Required Libraries](#installaiong)
* [Webscraping](#webscraping)
* [Data Cleaning](#cleaning)
* [Dataframe](#dataframe)
* [Mapping Toronto Postal Code](#map)
* [Analyzing Data Using K-Means Clustering](#cluster)

In this assignment, we will explore and cluster the neighborhoods in Toronto. The dataset is not readily available and we will be scraping it on Wikipedia, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M.

## Installing Required Libraries <a name="installation"></a>

For webscraping, we will be utilizing **BeautifulSoup** package. Our chosen parser is **lxml** and we will install **html5** as well to parse data from a website without any problem.

In [ ]:
!pip install BeautifulSoup4
!pip install lxml
!pip install html5

We will import required libraries. BeautifulSoup for webscraping, requests for GET request to a webpage, and pandas so we can prepare the data.

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

The following code is executed to source and html file is converted to text file. We then create the **BeautifulSoup** object, passing the source.

## Webscraping <a name="webscraping"></a>

In [ ]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'lxml')


We will now define our column names. By inspecting the website, headers of the table is under **'th'** tag. The **'th'** tag is passed to the **find_all** method to create a list under **'th'** tag. Slicing the first 3 gives us **'Postcode', 'Borough'**, and **'Neighbourhood'.**

In [ ]:
columns = [] # Column values

for header in soup.find_all('th')[0:3]:
    header = header.text
    columns.append(header)

columns[-1] = columns[-1].strip() # Removing the \n in the last item

Preparing the data. The rows list is created to contain the information. From the website, the data are located under **'tr'** tag. Passing to the **find_all** method. 

In [ ]:
# Rows values
rows = [] # Rows value container

for data in soup.find_all('tr')[1:]:
    data = data.text
    data = data.split('\n')
    rows.append(data)

rows = rows[0:-5] # Remove unwanted information
    
for i in range(len(rows)): # Clean data by removing the whitespace in the list
    rows[i] = rows[i][1:4]

rows

## Data Cleaning <a name="cleaning"></a>

The columns and rows are now established and the dataframe is will be created.

In [ ]:
df = pd.DataFrame(data=rows, columns=columns) 
df

Items that is 'Not assigned' in  **Borough** column is dropped.

In [ ]:
df = df[df.Borough != 'Not assigned'].reset_index()
df = df.drop(['index'], axis=1) # redundant index column is removed

**Neighbourhood** with the same **Postalcode** is grouped and concatenated.

In [ ]:
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df

The **Neighbourhood** with 'Not assigned' value will be replaced by the value in **Borough** column.

In [ ]:
df_not_assigned = df[df['Neighbourhood'].str.contains("Not assigned")] # Extracting the dataframe whose Neighbourhood is 'Not assigned'

for i in list(df_not_assigned['Borough']): # 'Not assigned' value is replaced
    df = df.replace('Not assigned', i)

## Dataframe <a name="dataframe"></a>

First 12 items in the dataframe are shown below.

In [ ]:
df.head(12)

The shape of the dataframe is shown.

In [ ]:
df.shape

## Coordinates

Importing the required libraries.

The **latitude** and **longitude** of every Postcode will be obtained by using Geocoder library. Since the Geocoder package is not working properly, then the downloaded csv file of latitude and longitude will be used.

In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

In [ ]:
#!pip install geocoder

In [ ]:
#import geocoder # import geocoder

# initialize your variable to None
#lat_lng_coords = None

# loop until you get the coordinates
#for postal_code in df['Postcode']:
#    while(lat_lng_coords is None):
#        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#        lat_lng_coords = g.latlng

#    df['Latitude'] = lat_lng_coords[0]
#    df['Longitude'] = lat_lng_coords[1]

The Geocoder is not working properly. Csv file of the coordinates will be downloaded and converted to dataframe as **df_loc.**

In [ ]:
!wget -q -O 'Geospatial_Coordinates.csv' https://cocl.us/Geospatial_data

In [ ]:
df_loc = pd.read_csv('Geospatial_Coordinates.csv')

We will reference the **Postal Codes** in our dataset to add latitude and longitude in our **df** dataframe.

In [ ]:
for post_code in df_loc['Postal Code']:
    post_code == df['Postcode']
    df['Latitude'] = df_loc['Latitude']
    df['Longitude'] = df_loc['Longitude']

In [ ]:
df

## Mapping Toronto Postal Code <a name="map"></a>

We will now map all places in Toronto based on the Postal Code.

In [ ]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

In [ ]:
map_of_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng], 
                        radius=5, 
                        popup=label,
                        color='blue',
                        fill=True,
                        fill_color='#3186cc',
                        fill_opacity=0.7,
                        parse_html=False).add_to(map_of_toronto)
    
map_of_toronto

## Analyzing Borough that has a name Toronto <a name="analyze"></a>

In this section, we will segment and cluster the neighbourhood under a borough that has a name Toronto. We will use FourSquare API to extract venue information and category to proceed to Analysis.

In [ ]:
borough_named_toronto = df[df['Borough'].str.contains('Toronto')].reset_index(drop=True)
borough_named_toronto

Now we will use the **FourSquare API** to extract data of the venues, and venue category, venue location in terms of latitude and longitude.

### FourSquare API credential

In [ ]:
CLIENT_ID = 'IIIODSQOYJSCWYL5N0DPK2HHWKSCHFSMAPV2AWSQ0KSURL21' # your Foursquare ID
CLIENT_SECRET = 'VBF3SZ4LR4KHHEU10SU4NQIATY42ETEKAJBEMOTVKT3XXWXG' # your Foursquare Secret
VERSION = '20200101' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Getting nearby venues using FourSquare API. We used the function defined to extract the Venue, Venue Category, and Location from the JSON file.

In [ ]:
LIMIT = 100


def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

The new dataset contains table such as  Neighbourhood, Neighbourhood Latitude, Neighbourhood Longitude, Venue, Venue Latitude, Venue Longitude, and Venue Category named as **toronto_venues**.

In [ ]:
toronto_venues = getNearbyVenues(names=borough_named_toronto['Neighbourhood'],
                                   latitudes=borough_named_toronto['Latitude'],
                                   longitudes=borough_named_toronto['Longitude']
                                  )

In [ ]:
toronto_venues.head()

In [ ]:
toronto_venues.groupby('Neighbourhood').count()

Checking the shape of our defined dataset.

In [ ]:
toronto_venues.shape

In [ ]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

### Analyzing Data

We will convert all the categorical data, which is the **Venue Category** in our dataset to numerical using one hot encoding.

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

The data are grouped in terms of Neighbouhood and get its mean from the number of occurence in a particular category.

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

Checking the number of rows and columns

In [ ]:
toronto_grouped.shape

The top 5 categories in each Neighbourhood is shown using the following code.

In [ ]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

The data show the frequency in terms of mean of every venue category. It is clearly shown the most popular venue category in every Neighbourhood or group of Neighboorhoods.

Now we will create a table ranking the top 10 most common venue in every Neighbourhood or group of Neighbourhoods.

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

## Analyzing Data Using K-Means Clustering <a name="cluster"></a>

We will now cluster our data using K-Means clustering. Defining our number of k as 5.

In [ ]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

We will create a table appending our **kmeans.labels_** output.

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = borough_named_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() 

## Mapping the Clusters

We used **folium** to create a map and put a circle marker and labels on the map. 

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

We will extract every cluster and let's see what conclusions can be drawn.

#### Cluster 1

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

#### Cluster 2

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

#### Cluster 3

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

#### Cluster 4

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

#### Cluster 5

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

## Observation and Conclusion

Under the assumption that venues related to food and dining are mostly where people meet. Given that the features used in clustering of these data are their most common venues. Looking at the 1st most common venue in every cluster. Cluster 1 most common venue are ralated to food and dining, cafe or coffee shops. It shows that cluster 1 has places where many people meet. Cluster 2, 3, 4, and 5 most common venues are garden, playground, park, Dimsum Restaurant and neighborhood. We can say that these clusters have lesser people visiting their most common venues.